# Train and Deploy YOLOV5 Classification on Sagemaker
This notebook shows the steps needed to train and deploy a YOLOV5 classification model on Amazon Sagemaker

## Step 0. Import needed libraries

In [ ]:
!pip install shortuuid

In [ ]:
import json
import numpy as np
import pandas as pd
import os
import boto3
import sagemaker
import uuid
import time
import shortuuid
import shutil
#import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline 
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.session import TrainingInput
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
from sagemaker.serializers import DataSerializer
from sagemaker.deserializers import JSONDeserializer
sm_session = sagemaker.Session()
role = get_execution_role()
s3_resource = boto3.resource('s3')

## Step 1. Training a Classification Model on Amazon SageMaker

### Clone the YOLO-V5 repository

In [ ]:
!git clone https://github.com/ultralytics/yolov5 

### Locate your training data

Training data should follow the following folder structure and only contain images:

Folder structure is in alphabetical order, remember to complete your classes array in the same order.

```
dataset
├── train
│   ├── class-one
│   │   ├── IMG_123.jpg
│   └── class-two
│       ├── IMG_456.jpg
├── val
│   ├── class-one
│   │   ├── IMG_789.jpg
│   └── class-two
│       ├── IMG_101.jpg
```

In [ ]:
classes = ["front", "frontleft", "frontright"]

If you are uploading your dataset to Amazon SageMaker Studio remember to delete any checkpoint files before uploading the dataset to Amazon S3:

In [ ]:
def delete_ipynb_checkpoints(root_directory):
    for foldername, subfolders, filenames in os.walk(root_directory):
        for subfolder in subfolders:
            if subfolder == ".ipynb_checkpoints":
                folder_path = os.path.join(foldername, subfolder)
                shutil.rmtree(folder_path)
                print(f"Deleted: {folder_path}")

# Replace the path with the actual path to the dataset directory where you want to start deleting .ipynb_checkpoints folders.
delete_ipynb_checkpoints('dataset')

In [ ]:
sm_session = sagemaker.Session()
bucket = sm_session.default_bucket()
dataset_s3_uri = sm_session.upload_data("dataset", bucket, "yolov5cars")
print("Dataset located in: ",dataset_s3_uri)

### Configure your Training Job

In [ ]:
training_name = "yolov5-train"

# Create a short unique identifier
short_id = shortuuid.uuid()

# Create job_name by combining training_name with the shorter unique identifier
job_name = '{}-{}'.format(training_name, short_id)

print(job_name)

In [ ]:
train_input = TrainingInput(dataset_s3_uri)
print("Your training input is located in:",dataset_s3_uri)

In [ ]:
hyperparameters={
    "workers":"8",
    "device": "0",
    "batch-size": "16",
    "epochs": 100,
    "data": "/opt/ml/input/data/training",
    "model": "yolov5s-cls.pt",
    "project": "/opt/ml/model"
}

estimator = PyTorch(
    framework_version='1.11.0',
    py_version='py38',
    entry_point='classify/train.py',
    source_dir='yolov5',
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    role=role,
    disable_profiler=True, 
    debugger_hook_config=False
)

In [ ]:
estimator.fit(train_input, job_name=job_name)

### Deploy the model to an endpoint
Define the model name and the model location

In [ ]:
model_name = "Model-"+job_name
model_data = 's3://{}/{}/output/model.tar.gz'.format(sm_session.default_bucket(), job_name)
print(model_data)

Configure the PyTorchModel

In [ ]:
model = PyTorchModel(
    entry_point='inference.py',
    source_dir='./helper-code',
    model_data=model_data,
    framework_version='1.11.0',
    py_version='py38',
    role=role,
    name=model_name
)

Deploy the model and define serializers

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.large')
predictor.deserializer = JSONDeserializer()
predictor.serializer =DataSerializer(content_type="image/png")

### Test your endpoint

In [ ]:
def print_class_and_percentage(probabilities, class_names):
    # Get the index of the maximum probability
    max_index = probabilities[0].index(max(probabilities[0]))
    
    # Get the class name corresponding to the maximum probability
    predicted_class = class_names[max_index]
    
    # Get the percentage (probability) corresponding to the predicted class
    confidence_percentage = max(probabilities[0]) * 100
    
    # Print the predicted class and its confidence percentage
    print(f'Class: {predicted_class}, Confidence Percentage: {confidence_percentage:.2f}%')

# Directory containing the images
image_directory = "test-images"

# List all files in the directory
files = os.listdir(image_directory)

# Filter image files (assuming image files have extensions like .jpg, .jpeg, .png)
image_files = [file for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Iterate over each image and print the class with the highest percentage
for image_file in image_files:
    image_path = os.path.join(image_directory, image_file)
    image_data = open(image_path, 'rb').read()
    
    # Get the model prediction
    results = json.loads(predictor.predict(image_data))
    
    # Print only the class with the highest percentage
    print(f"Results for {image_file}:")
    print_class_and_percentage(results, classes)
    print("----------------------")
